# Microestructuras y sistemas de Trading
## Laboratorio 1

## Equipo C
### Participantes:
>Paulina Milenka Glez Manrique, Andres García Ruvalcaba, Mariana López Correa,Juan López Pelayo.

In [45]:
import quandl 
import pandas as pd
import os
import calendar
import numpy as np

In [46]:
quandl.ApiConfig.api_key = "6ARMqMT_1jbzJ71XyRCx"

In [47]:
path=os.getcwd()
files=os.listdir(path)
etf_files = [f for f in files if f[-3:]=='csv']

In [48]:
df=[]
for f in etf_files: 
    data=pd.read_csv(f, header=0, sep=',', index_col=0, parse_dates=False, skip_blank_lines=True, skiprows=10)

    df.append(data)

In [49]:
df_nassets = []
for f in etf_files:
    data=pd.read_csv(f,nrows=4)
    data=data.iloc[3,0]
    df_nassets.append(data)

In [50]:
nassets = []
for i in range(len(df_nassets)):
    a=df_nassets[i]
    a=a.replace('.00','').replace(',','')
    a=int(a)
    nassets.append(a)

In [51]:
df_fechas=[]
for f in etf_files:
    data=pd.read_csv(f,nrows=4)
    data=data.iloc[1,0]
    df_fechas.append(data)

In [52]:
dic = {'abr':'04','ago':'08','dic':'12','feb':'02','ene':'01','jun':'06','jul':'07','mar':'03','may':'05','nov':'11','oct':'10','sep':'09'}
lenfechas=len(df_fechas)
for i in range(lenfechas):
    df_fechas[i]=df_fechas[i].replace(df_fechas[i][3:6],dic.get(df_fechas[i][3:6]))

In [53]:
l=len(df_fechas)
for i in range(l):
    df_fechas[i]=df_fechas[i][-4:]+df_fechas[i][2:6]+df_fechas[i][0:2]
df_fechas

['2017-02-28',
 '2017-03-31',
 '2017-04-28',
 '2017-05-31',
 '2017-06-30',
 '2017-07-31',
 '2017-08-31',
 '2017-09-29',
 '2017-10-31',
 '2017-11-30',
 '2017-12-29',
 '2018-01-31',
 '2018-02-28']

In [54]:
M1= df[0]
tickers = M1.index.get_values()
tickers = tickers.tolist()
prices = quandl.get_table('WIKI/PRICES',ticker = tickers,  qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
                       date = { 'gte': df_fechas[0], 'lte': df_fechas[-1] }, 
                       paginate=True)
prices.head()

C:\Users\milen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.
  


,ticker,date,adj_close
None,,,
0,ZTS,2018-02-28,80.86
1,ZTS,2018-02-27,81.67
2,ZTS,2018-02-26,83.30
3,ZTS,2018-02-23,82.79
4,ZTS,2018-02-22,80.76


In [55]:
df_prices = []
for i in range(len(df)-1):
    arch= df[i]
    tickers = arch.index.get_values()
    tickers = tickers.tolist()
    prices = quandl.get_table('WIKI/PRICES',ticker = tickers,  qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
                       date = { 'gte': df_fechas[i], 'lte': df_fechas[-1] }, 
                       paginate=True)
    df_prices.append(prices)

C:\Users\milen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.
  after removing the cwd from sys.path.


In [56]:
p_prices=prices.pivot('ticker','date','adj_close')
#p_prices=p_prices.fillna(p_prices.mean(axis=1))
p_prices=p_prices.fillna(method='ffill',axis=1)

In [57]:
q_acc=[]
l=len(df)
for i in range(l):
    data=df[i]['Acciones'].dropna()
    data=data.sort_index(ascending=True)
    q_acc.append(data)
q_acc[0].head()

Ticker
A         308,935.00
AAP       107,750.00
AAPL    7,770,569.00
ABBV    2,368,365.00
ACN       904,323.00
Name: Acciones, dtype: object

In [70]:
acciones=[]
l=len(q_acc)
for j in range(l):
    lista=pd.DataFrame()
    l=len(q_acc[j])
    for i in range(l):
        data=q_acc[j].to_frame()
        #data=data.iloc[i].replace(',','')
        lista=lista.append(data)
    acciones.append(lista)


In [71]:
l=len(acciones)
acc_clean = []
for i in range(l):
    a=acciones[i]
    a=a['Acciones'].str.replace('.00','').str.replace(',','').astype(int)
    acc_clean.append(a)


ValueError: invalid literal for int() with base 10: ''

In [60]:
all_data = p_prices.join(acc_clean[0])


In [61]:
fechas=all_data.columns.get_values()
fechas=fechas[0:-1]

C:\Users\milen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.
  """Entry point for launching an IPython kernel.


In [62]:
precios=all_data.iloc[:,0:-1].transpose()
cantidades=all_data['Acciones']

In [63]:
totales=np.dot(precios,cantidades)
totales=pd.DataFrame(totales,index=fechas)

In [64]:
navs_diarios=totales/nassets[0]

In [65]:
rends=[]
for i in range(len(df_fechas)-1):
    rend=rend1=navs_diarios.transpose()[df_fechas[i+1]]/navs_diarios.transpose()[df_fechas[i]]-1
    rends.append(rend)
rends=pd.DataFrame(rends,index=df_fechas[1:])

KeyError: '2017-03-31'

In [66]:
plt.figure(figsize=(20,6))
plt.plot(rends, marker='o',)
plt.xlabel("fechas")   
plt.ylabel("rendimientos")
plt.show()

NameError: name 'plt' is not defined

In [67]:
rendimiento_final=navs_diarios.iloc[-1]/navs_diarios.iloc[0]-1

In [68]:
navs_diarios=pd.DataFrame(navs_diarios)

In [69]:
rends_diarios=((navs_diarios-navs_diarios.shift(1))/navs_diarios.shift(1)).iloc[1:]